<a href="https://colab.research.google.com/github/Migro-oea/Supervised_Learning-/blob/main/Binary_prediction_on_Rainfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [49]:
train = pf.read_csv('train.csv')
test = pf.read_csv('test.csv')
ss = pf.read_csv('sample_submission.csv')

In [50]:
ss

,id,rainfall
0,2190,0
1,2191,0
2,2192,0
3,2193,0
4,2194,0
...,...,...
725,2915,0
726,2916,0
727,2917,0
728,2918,0


In [51]:
ss.shape

(730, 2)

In [52]:
train.head()

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


In [53]:
train.isnull().sum()

,0
id,0
day,0
pressure,0
maxtemp,0
temparature,0
mintemp,0
dewpoint,0
humidity,0
cloud,0
sunshine,0


In [54]:
# Create a Target Column
train['rainfall'] = np.where(train['rainfall']>0,1,0)

In [55]:
train['rainfall']

,rainfall
0,1
1,1
2,1
3,1
4,0
...,...
2185,1
2186,1
2187,1
2188,1


In [56]:
X = train.drop(['rainfall'], axis=1)
y = train['rainfall']
X_test_full = test.copy()

In [57]:
X = X.drop(['id', 'day'], axis=1)
X_test = X_test_full.drop(['id', 'day'], axis=1)

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
#--Define Model and Hyperparameter Grid--
rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

In [60]:
# Sclaing + Logistic regression

model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

In [61]:
# Trainig the Model
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression())])

In [62]:
# Build Pipeline + GridSearch
from sklearn.preprocessing import StandardScaler
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', rf)
])

param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2],
    'classifier__max_features': ['sqrt', 'log2']
}


grid_search = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    cv = 5,
    n_jobs = -1,
    scoring = 'accuracy',
    return_train_score = True
)

grid_search.fit(X_train, y_train)
print(f'Best Parameters: {grid_search.best_params_}')

Best Parameters: {'classifier__max_depth': 10, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}


In [63]:
# Evaluation on validation set
Best_model = grid_search.best_estimator_
y_val_pred = Best_model.predict(X_val)
print(f'\nValiation Accuracy: {accuracy_score(y_val, y_val_pred)}')
print(f'\nClassification Report:\n', {classification_report(y_val, y_val_pred)})


Valiation Accuracy: 0.8515981735159818

Classification Report:
 {'              precision    recall  f1-score   support\n\n           0       0.79      0.61      0.69       119\n           1       0.87      0.94      0.90       319\n\n    accuracy                           0.85       438\n   macro avg       0.83      0.78      0.80       438\nweighted avg       0.85      0.85      0.85       438\n'}


In [64]:
# Inspect Cross-Validation Result
cv_results = pd.DataFrame(grid_search.cv_results_)
print("\nCross-Validation Performance:\n", cv_results[['mean_test_score', 'std_test_score', 'mean_train_score']].head())


Cross-Validation Performance:
    mean_test_score  std_test_score  mean_train_score
0         0.871013        0.010939          0.999572
1         0.869867        0.009078          1.000000
2         0.871583        0.010892          1.000000
3         0.870439        0.012672          0.981021
4         0.872153        0.012095          0.980165


In [66]:
test_prediction = Best_model.predict(X_test)

In [68]:
output = pd.DataFrame({'id': test['id'], 'rainfall': test_prediction})

print("\nSample Predictions:\n", output.head())


Sample Predictions:
      id  rainfall
0  2190         1
1  2191         1
2  2192         1
3  2193         0
4  2194         0


In [69]:
# Save Predictions
output.to_csv('rainfall_prediction.csv', index=False)
print("\nPredictions saved to 'rainfall_prediction.csv'")


Predictions saved to 'rainfall_prediction.csv'


In [70]:
rp = pd.read_csv('rainfall_prediction.csv')

In [71]:
rp

,id,rainfall
0,2190,1
1,2191,1
2,2192,1
3,2193,0
4,2194,0
...,...,...
725,2915,1
726,2916,1
727,2917,1
728,2918,1
